# Python

In [6]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date

def get_python_latest_stable():
    url = "https://www.python.org/downloads/"
    resp = requests.get(url)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")
    # Find the "latest" release box. Python.org typically has:
    # <span class="release-number">Latest Python 3 Release - Python 3.12.1</span>
    release_number_span = soup.find("span", class_="release-number")
    latest_text = release_number_span.get_text(strip=True)  # e.g. "Latest Python 3 Release - Python 3.12.1"

    # Extract version from string
    # Typically: "Latest Python 3 Release - Python 3.12.1"
    # We'll split on "Python" and take the last part:
    version_str = latest_text.split("Python")[-1].strip()   # "3.12.1"

    # If we want release date:
    release_date_span = soup.find("span", class_="release-date")
    # Typically: "October  2, 2023"
    date_text = release_date_span.get_text(strip=True) if release_date_span else None
    if date_text:
        try:
            release_date = datetime.strptime(date_text, "%B %d, %Y").date()
        except ValueError:
            release_date = None
    else:
        release_date = None

    return version_str, release_date

print(get_python_latest_stable())

('Release version', None)


In [1]:

import requests
from datetime import datetime, date


def get_python_latest_stable() -> tuple[str | None, date | None]:
    url = "https://api.github.com/repos/python/cpython/releases"
    headers = {"Accept": "application/vnd.github.v3+json"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    releases = response.json()
    for release in releases:
        if not release["prerelease"] and not release["draft"]:
            version = release["tag_name"].lstrip("v")  # Remove 'v' prefix
            date_str = release["published_at"]
            release_date = datetime.fromisoformat(date_str.replace("Z", "+00:00")).date()
            return version, release_date
    return None, None

print(get_python_latest_stable())

(None, None)


# Java

In [3]:
from datetime import datetime, date

def get_java_latest_stable() -> tuple[str | None, date | None]:
    url = "https://api.github.com/repos/adoptium/temurin21-binaries/releases"
    headers = {"Accept": "application/vnd.github.v3+json"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    releases = response.json()
    for release in releases:
        if not release["prerelease"] and not release["draft"]:
            version = release["tag_name"].replace("jdk-", "").replace("+", ".")
            date_str = release["published_at"]
            release_date = datetime.fromisoformat(date_str.replace("Z", "+00:00")).date()
            return version, release_date
    return None, None

print(get_java_latest_stable())

('21.0.6.7', datetime.date(2025, 1, 22))


# Maven

In [22]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime

def fetch_maven_version_info(group_id: str, artifact_id: str) -> tuple[str, datetime]:
    """
    Fetch the latest version and release date from Maven Central for the given artifact.
    """
    # Convert group_id dots to slashes for URL
    group_path = group_id.replace('.', '/')
    
    url = f"https://repo1.maven.org/maven2/{group_path}/{artifact_id}/maven-metadata.xml"
    response = requests.get(url)
    response.raise_for_status()
    
    # Parse XML
    root = ET.fromstring(response.content)
    
    # Get latest version
    versioning = root.find('versioning')
    if versioning is None:
        raise ValueError(f"No versioning info found for {group_id}:{artifact_id}")
        
    latest = versioning.find('latest')
    if latest is None:
        raise ValueError(f"No latest version found for {group_id}:{artifact_id}")
    
    latest_version = latest.text

    if latest_version is None:
        raise ValueError(f"No latest version found for {group_id}:{artifact_id}")
    
    # Get release date from lastUpdated
    last_updated = versioning.find('lastUpdated')
    if last_updated is None:
        raise ValueError(f"No lastUpdated found for {group_id}:{artifact_id}")
    
    if last_updated.text is None:
        raise ValueError(f"lastUpdated is empty for {group_id}:{artifact_id}")
        
    # Convert Maven timestamp (yyyyMMddHHmmss) to datetime
    timestamp = last_updated.text
    release_dt = datetime.strptime(timestamp, '%Y%m%d%H%M%S')
    
    return latest_version, release_dt

# Example usage
group_id = "org.spkringframework.boot"
artifact_id = "spring-boot-starter-parent"

version, date = fetch_maven_version_info(group_id, artifact_id)
print(f"Latest version: {version}")
print(f"Release date: {date}")

HTTPError: 404 Client Error: Not Found for url: https://repo1.maven.org/maven2/org/spkringframework/boot/spring-boot-starter-parent/maven-metadata.xml

In [ ]:
from urllib.parse import urlencode
import requests
from datetime import datetime, UTC

def fetch_maven_release_date(group_id: str, artifact_id: str, version: str) -> datetime:
    """
    Returns the UTC datetime at which a specific Maven artifact version
    was published to Maven Central, using the search.maven.org API.
    """
    # Build the Solr-style query:
    #   q = g:"groupId" AND a:"artifactId" AND v:"version"
    #   core=gav (search only group/artifact/version data)
    #   rows=1 (just need the first doc)
    #   wt=json (we want JSON response)
    query = f'g:"{group_id}" AND a:"{artifact_id}" AND v:"{version}"'
    url = "https://search.maven.org/solrsearch/select"
    params = {
        "q": query,
        "core": "gav",
        "rows": 1,
        "wt": "json",
    }
    resp = requests.get(url, params=params)

    resp.raise_for_status()
    data = resp.json()

    docs = data.get("response", {}).get("docs", [])
    if not docs: # here raise LibraryVersionNotFoundError
        raise ValueError(
            f"Could not find artifact {group_id}:{artifact_id}:{version} on Maven Central"
        )
    
    doc = docs[0]
    assert doc["g"] == group_id, f"Group mismatch: {doc['g']} != {group_id}"
    assert doc["a"] == artifact_id, f"Artifact mismatch: {doc['a']} != {artifact_id}"
    assert doc["v"] == version, f"Version mismatch: {doc['v']} != {version}"
    
    # 'timestamp' is epoch milliseconds
    ts_millis = docs[0]["timestamp"]
    return datetime.fromtimestamp(ts_millis / 1000.0, UTC)


version = "3.4.3"
group_id = "org.springframework.boot"
artifact_id = "spring-boot-starter-parent"

dt = fetch_maven_release_date(
    group_id=group_id,
    artifact_id=artifact_id,
    version=version,
)
print(f"{group_id}:{artifact_id}:{version} published at: {dt.isoformat()}")


ValueError: Could not find artifact org.springframework.boot:spring-boot-starter-parent:3.4.3 on Maven Central

# C# / Nuget

In [4]:
import requests
from datetime import datetime, date

def get_dotnet_latest_stable() -> tuple[str, date ]:
    url = "https://api.github.com/repos/dotnet/core/releases"
    headers = {"Accept": "application/vnd.github.v3+json"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    releases = response.json()
    for release in releases:
        if not release["prerelease"] and not release["draft"] and release["tag_name"].startswith("v"):
            version = release["tag_name"].lstrip("v")
            date_str = release["published_at"]
            release_date = datetime.fromisoformat(date_str.replace("Z", "+00:00")).date()
            return version, release_date
    raise ValueError("No stable version found for dotnet")

print(get_dotnet_latest_stable())

('9.0.2', datetime.date(2025, 2, 11))


In [ ]:
https://api.nuget.org/v3/catalog0/index.json

# Ruby

In [ ]:
https://rubygems.org/api/v1/gems/<gem_name>.json

# Go

In [5]:
import requests
from datetime import datetime, date

def get_go_latest_stable() -> tuple[str, date | None]:
    url = "https://proxy.golang.org/golang.org/x/go/@v/list"
    response = requests.get(url)
    response.raise_for_status()
    versions = response.text.splitlines()
    stable_versions = [v.strip() for v in versions if not any(x in v for x in ["rc", "beta", "alpha"])]
    version = max(stable_versions, key=lambda v: tuple(map(int, v.lstrip("v").split("."))))
    return version.lstrip("v"), None  # Date requires additional scraping

print(get_go_latest_stable())

HTTPError: 404 Client Error: Not Found for url: https://proxy.golang.org/golang.org/x/go/@v/list

# PHP

In [ ]:
https://packagist.org/packages/<vendor>/<package_name>.json


# Rust

In [ ]:
https://crates.io/api/v1/crates/{crate_name}

# JavaScript

In [2]:
import requests
from datetime import datetime, date

def get_nodejs_latest_stable() -> tuple[str, date | None]:
    url = "https://nodejs.org/dist/index.json"
    response = requests.get(url)
    response.raise_for_status()
    releases = response.json()
    lts_releases = [r for r in releases if r["lts"] is not False]
    latest = max(lts_releases, key=lambda x: x["date"])
    version = latest["version"].lstrip("v")
    release_date = datetime.fromisoformat(latest["date"]).date()
    return version, release_date

print(get_nodejs_latest_stable())

('18.20.7', datetime.date(2025, 2, 20))
